# Bot Filosófico
---
## Primera fase
---
La primer fase de este bot de Twitter consiste en desarrollar algunas tareas básicas como retweetear o responder a otros usuarios. Para esto, buscamos desarrollar algoritmos que sean capaces de hacer búsquedas de palabras en la red social (el conjunto de palabras serán definidas en un array o un diccionario); cuando el bot encuentre alguna de estas palabras o frases va a activar la función de seleccionar una frase, esta frase será enviada a la API para que sea publicada como un tweet o, idealmente, como un retweet citado. 
El objetivo del bot es encontrar coinsidencias en el uso de las *trigger words* y su uso en algún texto filosófico.


Imoportamos algunos modulos y tokens de acceso a la API

In [ ]:
import tweepy, re, time, nltk
from access_tfbot import *
from random import randint

Definimos función para hacer peticiones a la API de Twitter

In [ ]:
# Setup API:
def twitter_setup():
	# Authenticate and access using keys:
	auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
	auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

	# Return API access:
	api = tweepy.API(auth)
	return api

Definimos función para leer libro de la data base "extract_status()" y llamamos función para seleccionar una frase

In [ ]:
def extract_status(path=None):
		if not path:
			return "No hay libro"

		#busca libro que leer y oracion 
		try: 
			with open(path, 'r', encoding='utf-8', errors="surrogateescape") as book:
				text = book.read()

			#si la lee exitosamente el libro, busca una oracion
			if text:
				return search_sentence(text)

		except:
			return "libro no encontrado"


Definimos la función para seleccionar frases "search_sentence()"

In [ ]:
def search_sentence(text): 
	status = 250

#mientras tengamos un status largo o muy corto
	while not (5 < status < 225):
	#genera un numero aleatorio
		index = randint(0, len(text))
	#determina indices de la oracion
		init_index = text[index:].find(".") + 2 + index
		last_index = text[init_index:].find(".") + 2 + index
		status = len(text[init_index:last_index])
	#remplaza cualquier caracter que no sea palabra y numero con espacios
	sentence = text[init_index:last_index]
	sentence = re.sub("\W\d", " ", sentence)
	return sentence

Definimos una función que busque una palabra en un tweet


In [ ]:
def search_word_tweet(api):
        #definimos conjunto de palabras a buscar
        #enviamos el indice aleatorio de un elemento del conjunto definido arriba
		setWords = ["technology", "tech", "technical", "information"]
		randomWord = randint(0,len(setWords)-1)
		randomWord = setWords[randomWord]
		#buscamos una palabra aleatoria definida anteriormente e iteramos este proceso n veces en count
        #devolvemos una lista con los tweets encontrados
		searchTweet = api.search(q=randomWord, count=1)
		#iteramos sobre la lista de tweets encontrados
        #definimos un patron con regex para tokenizar el tweet
        #imprimimos el numero de iteracion junto con el texto del tweet y el conjunto ordenado de los tokens del tweet mayores a 3 caracteres
		for tweet in searchTweet:
					url = f"https://twitter.com/{tweet.user.screen_name}/status/{tweet.id}"
					tweet = tweet.text
					pattern = r'''(?x)                 # set flag to allow verbose regexps
              (?:[A-Z]\.)+         # abbreviations, e.g. U.S.A.
              | \w+(?:-\w+)*       # words with optional internal hyphens
              | \$?\d+(?:\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
              | \.\.\.             # ellipsis
              | [][.,;"'?():-_`]   # these are separate tokens; includes ], [
'''
					tokens = nltk.regexp_tokenize(tweet, pattern)
					print(f"{count}, {tweet}")
					words = [word for word in tokens if len(word) > 3]
					orderedWords = sorted(set(words))
					print(randomWord, "\n", orderedWords)
					tweeting(randomWord,url)
                    
					return orderedWords, randomWord, url

Definimos la función para tokenizar el texto extraido de la data base y será enviado a tweetear

In [ ]:
def tweeting(randomWord,url):
		status = extract_status("texto.txt")
		#tokenizamos el texto a enviar mayor a tres caracteres"
		status_tokens = nltk.tokenize.word_tokenize(status)
		status_tokens = [word for word in status_tokens if len(word) > 3]
		print(status_tokens)
        #si la palabra a buscar está en el texto tokenizado seleccionado de la data base, enviar tweet como retweet citado
		if randomWord in status_tokens:
					bot.update_status(status +" " + url)
					print(f"{status} \n Tweet enviado!")
					print("funciona")
		else:
					while not randomWord in status_tokens:
						tweeting(randomWord, url)
          #break
          
						return status, status_tokens